In [ ]:
import numpy as np 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from scipy.stats import fisher_exact
import statsmodels.api as sm
from scipy.stats import norm

pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

In [ ]:
patients = pd.read_csv('Pats.csv')
patients.head(10)

In [ ]:
adm_source = pd.read_csv('ICUadmsource.csv')
adm_source.head(10)

In [ ]:
patients['age'].median()

In [ ]:
patients['age'].mean()

In [ ]:
patients[patients['deceased'] == 1]['age'].median()

In [ ]:
patients[patients['deceased'] == 1]['age'].mean()

In [ ]:
patients[patients['deceased'] == 0]['age'].median()

In [ ]:
patients[patients['deceased'] == 0]['age'].mean()

In [ ]:
deceased = patients[patients['deceased'] == 1]['age']
alive = patients[patients['deceased'] == 0]['age']

u_stat, p_value = mannwhitneyu(deceased, alive)
print(f'Mann-Whitney U test p-value: {p_value}')

In [ ]:
log_reg = sm.Logit(ytrain, Xtrain).fit() 
print(log_reg.summary())

In [ ]:
adm_source['admitted_from'] = adm_source['admitted_from'].replace({'after ED': 'after ED', 'direct admission': 'direct_admission', 'after hem/onc dep': 'after_hem/onc', 'after another department':'after_another'})

In [ ]:
adm_source['admitted_from'].value_counts()

In [ ]:
adm_source['admitted_from'].value_counts()/1085

In [ ]:
patients_wsource = pd.merge(patients, adm_source, on='ind')
patients_wsource['gender'] = patients_wsource['gender'].replace({'M': 0, 'F': 1})
patients_wsource = pd.get_dummies(patients_wsource, columns=['admitted_from'], dtype='int')
patients_wsource

In [ ]:
deceased = patients_wsource['deceased']
patients_wsource  = patients_wsource.drop(columns=['deceased'])
patients_wsource['deceased']=deceased
patients_wsource  = patients_wsource.drop(columns=['ind'])
patients_wsource

In [ ]:
patients_wsource.corrwith(patients_wsource['deceased']).round(2).sort_values(ascending=False)

In [ ]:
patients_pres = patients_wsource.copy()

In [ ]:
num_corr_p = patients_pres.corr().round(2)

# Correlation Matrix-Heatmap Plot
mask = np.zeros_like(num_corr)
mask[np.triu_indices_from(mask)] = True # optional, to hide repeat half of the matrix

f, ax = plt.subplots(figsize=(25, 15))
sns.set(font_scale=1.5) # increase font size

ax = sns.heatmap(num_corr_p, mask=mask, annot=True, annot_kws={"size": 12}, linewidths=.5, cmap="coolwarm", fmt=".2f", ax=ax) # round to 2 decimal places
ax.set_title("Correlation matrix", fontsize=20) # add title
plt.show()

In [ ]:
patients_pres  = patients_pres.drop(columns=['admitted_from_after_another'])
patients_pres

In [ ]:
num_corr_p = patients_pres.corr().round(2)

# Correlation Matrix-Heatmap Plot
mask = np.zeros_like(num_corr_p)
mask[np.triu_indices_from(mask)] = True # optional, to hide repeat half of the matrix

f, ax = plt.subplots(figsize=(25, 15))
sns.set(font_scale=1.5) # increase font size

ax = sns.heatmap(num_corr_p, mask=mask, annot=True, annot_kws={"size": 12}, linewidths=.5, cmap="coolwarm", fmt=".2f", ax=ax) # round to 2 decimal places
ax.set_title("Correlation matrix", fontsize=20) # add title
plt.show()

In [ ]:
model_patients_pres = sm.Logit(patients_pres['deceased'],
                         sm.add_constant(patients_pres[['age', 'gender', 
                                'admitted_from_after ED', 'admitted_from_after_hem/onc', 'admitted_from_direct_admission']]))
result_patients_pres = model_patients_pres.fit()
print(result_patients_pres.summary())